In [1]:
import pandas as pd
import os
from sqlalchemy import create_engine
from config import post_pass
from sqlalchemy.orm import Session
from sqlalchemy.ext.automap import automap_base
from flask import Flask, jsonify
from sqlalchemy.sql.expression import and_

In [2]:
connection_string = f"postgres:{post_pass}@localhost:5432/Project2"
engine = create_engine(f'postgresql://{connection_string}')

Base = automap_base()
Base.prepare(engine, reflect=True)

In [3]:
#set table variable
covid_case_death = Base.classes.covid_case_death
covid_vac = Base.classes.covid_vac
us_unemployment = Base.classes.us_unemployment
state_population = Base.classes.state_population


In [4]:
#vis 1
date = input("Enter a date")

session = Session(engine)
results = session.query(covid_case_death.submission_date, covid_case_death.state, covid_case_death.tot_cases, covid_case_death.tot_death, covid_vac.series_complete_pop_pct, state_population.population, us_unemployment.unemployment_rate)\
.outerjoin(covid_vac, covid_case_death.submission_date == covid_vac.submission_date)\
.outerjoin(state_population, covid_case_death.state == state_population.state)\
.outerjoin(us_unemployment, and_(covid_case_death.submission_date == us_unemployment.submission_date, covid_case_death.state == us_unemployment.state))\
.filter(covid_case_death.submission_date == date).order_by(covid_case_death.submission_date).all()
session.close()

vis_1 = []
for submission_date, state, tot_cases, tot_death, series_complete_pop_pct, population, unemployment_rate in results:
    results_dic = {}
    results_dic['submission_date'] = submission_date
    results_dic['state'] = state
    results_dic['tot_cases'] = tot_cases
    results_dic['tot_death'] = tot_death
    results_dic['cases_per_100'] = round((tot_cases / population) * 100, 2)
    results_dic['deaths_per_100'] = round((tot_death / population) * 100, 2)
    results_dic['series_complete_pop_pct'] = series_complete_pop_pct
    results_dic['population'] = population
    results_dic['unemployment_rate'] = unemployment_rate
        
    vis_1.append(results_dic)
    
#results
vis_1

Enter a date2020, 8, 1


[{'submission_date': datetime.date(2020, 8, 1),
  'state': 'AK',
  'tot_cases': 3138,
  'tot_death': 34,
  'cases_per_100': 0.43,
  'deaths_per_100': 0.0,
  'series_complete_pop_pct': None,
  'population': 736081.0,
  'unemployment_rate': 7.2},
 {'submission_date': datetime.date(2020, 8, 1),
  'state': 'AL',
  'tot_cases': 102500,
  'tot_death': 2351,
  'cases_per_100': 2.04,
  'deaths_per_100': 0.05,
  'series_complete_pop_pct': None,
  'population': 5030053.0,
  'unemployment_rate': 7.1},
 {'submission_date': datetime.date(2020, 8, 1),
  'state': 'AR',
  'tot_cases': 43173,
  'tot_death': 458,
  'cases_per_100': 1.43,
  'deaths_per_100': 0.02,
  'series_complete_pop_pct': None,
  'population': 3013756.0,
  'unemployment_rate': 6.5},
 {'submission_date': datetime.date(2020, 8, 1),
  'state': 'AZ',
  'tot_cases': 177075,
  'tot_death': 3747,
  'cases_per_100': 2.47,
  'deaths_per_100': 0.05,
  'series_complete_pop_pct': None,
  'population': 7158923.0,
  'unemployment_rate': 7.0},
 {'s

In [5]:
#vis 2
state = input("Enter a state")

session = Session(engine)
results = session.query(covid_case_death.submission_date, covid_case_death.state, covid_case_death.tot_cases, covid_case_death.tot_death).filter(covid_case_death.state == state_population.state, covid_case_death.state == state).order_by(covid_case_death.submission_date).all()
session.close()

#format data
vis_2 = []
for submission_date, state, tot_cases, tot_death, in results:
    results_dic = {}
    results_dic['submission_date'] = submission_date
    results_dic['state'] = state
    results_dic['cases_per_100'] = round((tot_cases / population) * 100, 2)
    results_dic['deaths_per_100'] = round((tot_death / population) * 100, 2)
    
    vis_2.append(results_dic)
    
#results
vis_2

Enter a stateCA


[{'submission_date': datetime.date(2020, 2, 1),
  'state': 'CA',
  'cases_per_100': 0.0,
  'deaths_per_100': 0.0},
 {'submission_date': datetime.date(2020, 3, 1),
  'state': 'CA',
  'cases_per_100': 0.16,
  'deaths_per_100': 0.05},
 {'submission_date': datetime.date(2020, 4, 1),
  'state': 'CA',
  'cases_per_100': 3.25,
  'deaths_per_100': 0.09},
 {'submission_date': datetime.date(2020, 5, 1),
  'state': 'CA',
  'cases_per_100': 11.14,
  'deaths_per_100': 0.44},
 {'submission_date': datetime.date(2020, 6, 1),
  'state': 'CA',
  'cases_per_100': 23.49,
  'deaths_per_100': 0.85},
 {'submission_date': datetime.date(2020, 7, 1),
  'state': 'CA',
  'cases_per_100': 52.07,
  'deaths_per_100': 1.23},
 {'submission_date': datetime.date(2020, 8, 1),
  'state': 'CA',
  'cases_per_100': 101.54,
  'deaths_per_100': 1.89},
 {'submission_date': datetime.date(2020, 9, 1),
  'state': 'CA',
  'cases_per_100': 128.24,
  'deaths_per_100': 2.57},
 {'submission_date': datetime.date(2020, 10, 1),
  'state':

In [6]:
#vis 3
state = input("Enter a state")

session = Session(engine)
results = session.query(covid_case_death.submission_date, covid_case_death.state, covid_case_death.tot_cases, covid_case_death.tot_death, state_population.population).filter(covid_case_death.state == state_population.state, covid_case_death.state == state).order_by(covid_case_death.submission_date).all()
session.close()

#format data
vis_3 = []
for submission_date, state, tot_cases, tot_death, population in results:
    results_dic = {}
    results_dic['submission_date'] = submission_date
    results_dic['state'] = state
    results_dic['tot_cases'] = tot_cases
    results_dic['tot_death'] = tot_death
    
    vis_3.append(results_dic)
    
#results
vis_3


Enter a stateGA


[{'submission_date': datetime.date(2020, 2, 1),
  'state': 'GA',
  'tot_cases': 0,
  'tot_death': 0},
 {'submission_date': datetime.date(2020, 3, 1),
  'state': 'GA',
  'tot_cases': 2,
  'tot_death': 0},
 {'submission_date': datetime.date(2020, 4, 1),
  'state': 'GA',
  'tot_cases': 4806,
  'tot_death': 154},
 {'submission_date': datetime.date(2020, 5, 1),
  'state': 'GA',
  'tot_cases': 26176,
  'tot_death': 1167},
 {'submission_date': datetime.date(2020, 6, 1),
  'state': 'GA',
  'tot_cases': 46501,
  'tot_death': 2074},
 {'submission_date': datetime.date(2020, 7, 1),
  'state': 'GA',
  'tot_cases': 84688,
  'tot_death': 2827},
 {'submission_date': datetime.date(2020, 8, 1),
  'state': 'GA',
  'tot_cases': 194466,
  'tot_death': 3825},
 {'submission_date': datetime.date(2020, 9, 1),
  'state': 'GA',
  'tot_cases': 283281,
  'tot_death': 5733},
 {'submission_date': datetime.date(2020, 10, 1),
  'state': 'GA',
  'tot_cases': 337302,
  'tot_death': 7063},
 {'submission_date': datetime.d

In [7]:
#vis 4
state = input("Enter a state")

session = Session(engine)
results = session.query(covid_vac.submission_date, covid_vac.state, covid_vac.series_complete_pop_pct).filter(covid_vac.state == state).order_by(covid_vac.submission_date).all()
session.close()

#format data
vis_4 = []
for submission_date, state, series_complete_pop_pct in results:
    results_dic = {}
    results_dic['submission_date'] = submission_date
    results_dic['state'] = state
    results_dic['series_complete_pop_pct'] = series_complete_pop_pct
    vis_4.append(results_dic)
    
#results
vis_4

Enter a stateNY


[{'submission_date': datetime.date(2021, 1, 1),
  'state': 'NY',
  'series_complete_pop_pct': 0.0},
 {'submission_date': datetime.date(2021, 2, 1),
  'state': 'NY',
  'series_complete_pop_pct': 0.0},
 {'submission_date': datetime.date(2021, 3, 1),
  'state': 'NY',
  'series_complete_pop_pct': 0.0},
 {'submission_date': datetime.date(2021, 4, 1),
  'state': 'NY',
  'series_complete_pop_pct': 17.3},
 {'submission_date': datetime.date(2021, 5, 1),
  'state': 'NY',
  'series_complete_pop_pct': 35.3},
 {'submission_date': datetime.date(2021, 6, 1),
  'state': 'NY',
  'series_complete_pop_pct': 47.0},
 {'submission_date': datetime.date(2021, 7, 1),
  'state': 'NY',
  'series_complete_pop_pct': 54.2},
 {'submission_date': datetime.date(2021, 8, 1),
  'state': 'NY',
  'series_complete_pop_pct': 57.2},
 {'submission_date': datetime.date(2021, 9, 1),
  'state': 'NY',
  'series_complete_pop_pct': 60.3},
 {'submission_date': datetime.date(2021, 10, 1),
  'state': 'NY',
  'series_complete_pop_pct':

In [8]:
#Vis 5
state = input("Enter a state")

session = Session(engine)
results = session.query(us_unemployment.submission_date, us_unemployment.state, us_unemployment.unemployment_rate).filter(us_unemployment.state == state).order_by(us_unemployment.submission_date).all()
session.close()

#format data
vis_5 = []
for submission_date, state, unemployment_rate in results:
    results_dic = {}
    results_dic['submission_date'] = submission_date
    results_dic['state'] = state
    results_dic['unemployment_rate'] = unemployment_rate
    vis_5.append(results_dic)
    
#results
vis_5

Enter a stateAK


[{'submission_date': datetime.date(2020, 1, 1),
  'state': 'AK',
  'unemployment_rate': 5.0},
 {'submission_date': datetime.date(2020, 2, 1),
  'state': 'AK',
  'unemployment_rate': 5.1},
 {'submission_date': datetime.date(2020, 3, 1),
  'state': 'AK',
  'unemployment_rate': 5.1},
 {'submission_date': datetime.date(2020, 4, 1),
  'state': 'AK',
  'unemployment_rate': 11.8},
 {'submission_date': datetime.date(2020, 5, 1),
  'state': 'AK',
  'unemployment_rate': 11.6},
 {'submission_date': datetime.date(2020, 6, 1),
  'state': 'AK',
  'unemployment_rate': 11.3},
 {'submission_date': datetime.date(2020, 7, 1),
  'state': 'AK',
  'unemployment_rate': 11.0},
 {'submission_date': datetime.date(2020, 8, 1),
  'state': 'AK',
  'unemployment_rate': 7.2},
 {'submission_date': datetime.date(2020, 9, 1),
  'state': 'AK',
  'unemployment_rate': 7.0},
 {'submission_date': datetime.date(2020, 10, 1),
  'state': 'AK',
  'unemployment_rate': 6.5},
 {'submission_date': datetime.date(2020, 11, 1),
  'sta

In [9]:
#vis 6
state = input("Enter a state")

session = Session(engine)
results = session.query(covid_vac.submission_date, covid_vac.state, covid_vac.series_complete_janssen, covid_vac.series_complete_moderna, covid_vac.series_complete_pfizer).filter(covid_vac.state == state, covid_vac.submission_date == "2021, 10, 1").order_by(covid_vac.submission_date).all()
session.close()

#format data
vis_6 = []
for submission_date, state, series_complete_janssen, series_complete_moderna, series_complete_pfizer in results:
    results_dic = {}
    total = series_complete_janssen + series_complete_moderna + series_complete_pfizer
    results_dic['submission_date'] = submission_date
    results_dic['state'] = state
    results_dic['series_complete_janssen'] = round((series_complete_janssen/total) * 100, 2)
    results_dic['series_complete_moderna'] = round((series_complete_moderna/total) * 100, 2)
    results_dic['series_complete_pfizer'] = round((series_complete_pfizer/total) * 100, 2)
    vis_6.append(results_dic)
    
#results
vis_6

Enter a stateNM


[{'submission_date': datetime.date(2021, 10, 1),
  'state': 'NM',
  'series_complete_janssen': 7.33,
  'series_complete_moderna': 39.99,
  'series_complete_pfizer': 52.68}]

In [13]:
print (vis_1)
print ("-"*50)
print (vis_2)
print ("-"*50)
print (vis_3)
print ("-"*50)
print (vis_4)
print ("-"*50)
print (vis_5)
print ("-"*50)
print (vis_6)

[{'submission_date': datetime.date(2020, 8, 1), 'state': 'AK', 'tot_cases': 3138, 'tot_death': 34, 'cases_per_100': 0.43, 'deaths_per_100': 0.0, 'series_complete_pop_pct': None, 'population': 736081.0, 'unemployment_rate': 7.2}, {'submission_date': datetime.date(2020, 8, 1), 'state': 'AL', 'tot_cases': 102500, 'tot_death': 2351, 'cases_per_100': 2.04, 'deaths_per_100': 0.05, 'series_complete_pop_pct': None, 'population': 5030053.0, 'unemployment_rate': 7.1}, {'submission_date': datetime.date(2020, 8, 1), 'state': 'AR', 'tot_cases': 43173, 'tot_death': 458, 'cases_per_100': 1.43, 'deaths_per_100': 0.02, 'series_complete_pop_pct': None, 'population': 3013756.0, 'unemployment_rate': 6.5}, {'submission_date': datetime.date(2020, 8, 1), 'state': 'AZ', 'tot_cases': 177075, 'tot_death': 3747, 'cases_per_100': 2.47, 'deaths_per_100': 0.05, 'series_complete_pop_pct': None, 'population': 7158923.0, 'unemployment_rate': 7.0}, {'submission_date': datetime.date(2020, 8, 1), 'state': 'CA', 'tot_case